In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import spacy
import sys
import gpt3
import cross_coref
import entity_linkage

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dgg32\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
error loading _jsonnet (this is expected on Windows), treating C:\Users\dgg32\AppData\Local\Temp\tmpheunj9e7\config.json as plain json
Some weights of the model checkpoint at microsoft/infoxlm-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

In [2]:
tokenizer = AutoTokenizer.from_pretrained("pucpr/clinicalnerpt-chemical")

model = AutoModelForTokenClassification.from_pretrained("pucpr/clinicalnerpt-chemical")

model_infer = pipeline('ner',model=model,tokenizer=tokenizer)

nlp = spacy.load("en_core_web_sm")

In [ ]:
verb_file = "cazy_keywords.txt"
verbs = open(verb_file).read().split("\n")
verbs

In [3]:
input_file = "./cazy/example_1.txt"

with open(input_file, 'r', newline="", encoding='utf-8') as infile:
    for line in infile:
        fields = line.strip().split("|")
        doi = fields[0]
        is_primary = fields[1]
        raw_text = fields[2]

        sentences = (cross_coref.resolve_pronoun(raw_text))

        doc = nlp(sentences)
        assert doc.has_annotation("SENT_START")
        for sent in doc.sents:
            has_verb = False
            for v in verbs:
                if v in sent.text.lower():
                    has_verb = True
            
            if has_verb == True:
            
                entities = model_infer(sent.text)
                #print (entities)
                if len(entities) > 0:
                    #print (f'"{sent.text.strip()}"')
                    res = gpt3.extract_relation("gpt3_training_cazy.txt", sent.text.strip() + "\n")
                    
                    #print(res)

                    results = res.split(",")

                    if len(results) == 3:
                        disam = entity_linkage.name_disambiguation(results[2])
                        results[2] = disam

                        print (",".join(results))


#Discovery and screening of novel metagenome-derived GH107 enzymes targeting sulfated fucans from brown algae

###Author

#####Soojin Kim, Hyuntak Nam, Seokpil Kang, Hyun-Seok Kim, Sangwook Park, Sang-Hee Ko, and Jinsoo Kim

#####Department of Chemical Engineering, Korea Advanced Institute of Science and Technology (KAIST), Daejeon, Republic of Korea

#####E-mail: jinsoo@kaist.ac.kr

This repository contains all the
